In [27]:
import altair as alt
import numpy as np
import pandas as pd

In [28]:
fl = pd.read_csv("/Users/pr158admin/Desktop/Finality Subsets/FL subsetfinalized.csv")
tx = pd.read_csv("/Users/pr158admin/Desktop/Finality Subsets/TX subsetfinalized.csv")
wa = pd.read_csv("/Users/pr158admin/Desktop/Finality Subsets/WA subsetfinalized.csv")

In [29]:
fl.head(10)

,BUYER_STATE_x,BUYER_COUNTY_x,TRANSACTION_YEAR,TRANSACTION_DATE_DT,Opioids_Shipment_IN_GM,fips,population,Deaths,opioid_shipment_population_ratio
0,FL,palm beach county,2006,2006-08-18,5.3790,12099.0,1284489.0,211.0,0.000004
1,FL,palm beach county,2006,2006-11-29,10.7580,12099.0,1284489.0,211.0,0.000008
2,FL,palm beach county,2007,2007-02-06,32.2740,12099.0,1286586.0,262.0,0.000025
3,FL,palm beach county,2007,2007-03-01,32.2740,12099.0,1286586.0,262.0,0.000025
4,FL,palm beach county,2007,2007-04-16,8.0685,12099.0,1286586.0,262.0,0.000006
5,FL,palm beach county,2007,2007-04-27,48.4110,12099.0,1286586.0,262.0,0.000038
6,FL,hernando county,2006,2006-11-03,3.0270,12053.0,164398.0,25.0,0.000018
7,FL,hernando county,2006,2006-12-04,3.0270,12053.0,164398.0,25.0,0.000018
8,FL,pinellas county,2007,2007-04-04,2.6895,12103.0,918624.0,235.0,0.000003
9,FL,pinellas county,2007,2007-04-12,5.3790,12103.0,918624.0,235.0,0.000006


In [34]:
# def dataset(df, year, state):
#     """make dataset ready to run diff-diff and pre-post for opioids"""
#     df["TRANSACTION_YEAR"] = df["TRANSACTION_YEAR"].astype("int")
#     opioids_data = (df.groupby(["BUYER_STATE_x", "TRANSACTION_YEAR"], as_index=False).sum())
#     opioids_data["policy"] = 0
#     opioids_data.loc[opioids_data["TRANSACTION_YEAR"] > year, "policy"] = 1
#     opioids_data["state"] = 0
#     opioids_data.loc[opioids_data["BUYER_STATE_x"] == state, "state"] = 1
#     df["death_rate"]= df["Deaths"] / df["population"]
#     return opioids_data

def dataset(df, year, state):
    """make dataset ready to run diff-diff and pre-post for opioids"""
    df["TRANSACTION_YEAR"] = df["TRANSACTION_YEAR"].astype("int")
    #opioids_data = (df.groupby(["BUYER_STATE_x", "TRANSACTION_YEAR",], as_index=False).sum())
    groupby_1 = (df.groupby(['BUYER_STATE_x', 'BUYER_COUNTY_x','TRANSACTION_YEAR','population','Deaths',],as_index=False)['opioid_shipment_population_ratio'].sum())
    groupby_1['Total shipment']= groupby_1['population']*groupby_1['opioid_shipment_population_ratio']
    opioids_data = (groupby_1.groupby(['BUYER_STATE_x','TRANSACTION_YEAR',],as_index=False)['population','Deaths','Total shipment'].sum())
    opioids_data["death_rate"]= opioids_data["Deaths"]/opioids_data["population"]
    opioids_data["opioid_shipment_population_ratio"]= opioids_data["Total shipment"]/opioids_data["population"]
    opioids_data["policy"] = 0
    opioids_data.loc[opioids_data["TRANSACTION_YEAR"] > year, "policy"] = 1
    opioids_data["state"] = 0
    opioids_data.loc[opioids_data["BUYER_STATE_x"] == state, "state"] = 1
    df["death_rate"]= df["Deaths"] / df["population"]
    return opioids_data



In [35]:
FL = dataset(fl, 2010, "FL")

/var/folders/5n/zdsqqtvs4ml7dvhr54rp68x00000gr/T/ipykernel_51812/4181839578.py:18: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  opioids_data = (groupby_1.groupby(['BUYER_STATE_x','TRANSACTION_YEAR',],as_index=False)['population','Deaths','Total shipment'].sum())


In [36]:
FL.head(10)

,BUYER_STATE_x,TRANSACTION_YEAR,population,Deaths,Total shipment,death_rate,opioid_shipment_population_ratio,policy,state
0,FL,2006,18154693.0,2303.0,7.572175e+06,0.000127,0.417092,0,1
1,FL,2007,18355485.0,2546.0,9.275845e+06,0.000139,0.505345,0,1
2,FL,2008,18514553.0,2660.0,1.207714e+07,0.000144,0.652305,0,1
3,FL,2009,18222420.0,2795.0,1.541160e+07,0.000153,0.845750,0,1
4,FL,2010,18511620.0,2748.0,1.999628e+07,0.000148,1.080202,0,1
5,FL,2011,18688787.0,2667.0,1.518623e+07,0.000143,0.812585,1,1
6,FL,2012,18885152.0,2299.0,9.429126e+06,0.000122,0.499288,1,1
7,FL,2013,19091156.0,2180.0,7.391156e+06,0.000114,0.387151,1,1
8,FL,2014,19361792.0,2448.0,7.102058e+06,0.000126,0.366808,1,1
9,HI,2006,1309622.0,58.0,2.482951e+05,0.000044,0.189593,0,0


In [37]:
TX = dataset(tx, 2007, "TX")

/var/folders/5n/zdsqqtvs4ml7dvhr54rp68x00000gr/T/ipykernel_51812/4181839578.py:18: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  opioids_data = (groupby_1.groupby(['BUYER_STATE_x','TRANSACTION_YEAR',],as_index=False)['population','Deaths','Total shipment'].sum())


In [52]:
TX.head(10)

,BUYER_STATE_x,TRANSACTION_YEAR,population,Deaths,Total shipment,death_rate,opioid_shipment_population_ratio,policy,state
0,CA,2006,36021202.0,3207.0,7.309936e+06,0.000089,0.202934,0,0
1,CA,2007,36249037.0,3614.0,8.338893e+06,0.000100,0.230045,0,0
2,CA,2008,36604337.0,3555.0,9.178293e+06,0.000097,0.250743,1,0
3,CA,2009,36308527.0,3782.0,1.014278e+07,0.000104,0.279350,1,0
4,CA,2010,36637290.0,3773.0,1.051896e+07,0.000103,0.287111,1,0
5,CA,2011,36969200.0,3948.0,1.098462e+07,0.000107,0.297129,1,0
6,CA,2012,37325068.0,3780.0,1.139486e+07,0.000101,0.305287,1,0
7,CA,2013,37658016.0,4223.0,1.108976e+07,0.000112,0.294486,1,0
8,CA,2014,38065718.0,4292.0,1.061830e+07,0.000113,0.278947,1,0
9,CO,2006,4712675.0,481.0,1.000047e+06,0.000102,0.212204,0,0


In [38]:
WA = dataset(wa, 2012, "WA")

/var/folders/5n/zdsqqtvs4ml7dvhr54rp68x00000gr/T/ipykernel_51812/4181839578.py:18: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  opioids_data = (groupby_1.groupby(['BUYER_STATE_x','TRANSACTION_YEAR',],as_index=False)['population','Deaths','Total shipment'].sum())


In [39]:
def pre_post(data, yvar, xvar, year,  alpha=0.05):
    import statsmodels.formula.api as smf

    # Grid for predicted values
    data1 = data.loc[(data["policy"]==0) & (data["state"] ==1), :]
    x = data1.loc[pd.notnull(data1[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions1 = pd.DataFrame({xvar: grid})

    # Fit model_before, get predictions
    model_before = smf.ols(f"{yvar} ~ {xvar}", data=data1).fit()
    model_predict = model_before.get_prediction(predictions1[xvar])
    predictions1[yvar] = model_predict.summary_frame()["mean"]
    predictions1[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=alpha)

    # Build chart
    reg1 = alt.Chart(predictions1).mark_line().encode(x=xvar, y=yvar)
    ci1 = (
        alt.Chart(predictions1)
        .mark_errorband()
        .encode(
            x=xvar,
            y=alt.Y("ci_low", title=yvar),
            y2="ci_high",
        )
    )
    
    data2 = data.loc[(data["policy"]==1) & (data["state"]==1),:]
    x = data2.loc[pd.notnull(data2[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions2 = pd.DataFrame({xvar: grid})

     # Fit model_before, get predictions
    model_after = smf.ols(f"{yvar} ~ {xvar}", data=data2).fit()
    model_predict_after = model_after.get_prediction(predictions2[xvar])
    predictions2[yvar] = model_predict_after.summary_frame()["mean"]
    predictions2[["ci_low", "ci_high"]] = model_predict_after.conf_int(alpha=alpha)

    # Build chart
    reg2 = alt.Chart(predictions2).mark_line().encode(x=xvar, y=yvar)
    ci2 = (
        alt.Chart(predictions2)
        .mark_errorband()
        .encode(
            x=xvar,
            y=alt.Y("ci_low", title=yvar),
            y2="ci_high",
        )
    )

    overlay = pd.DataFrame({'x': [year]})
    vline = alt.Chart(overlay).mark_rule(color='red', strokeWidth=3).encode(x='x:Q')
    
    chart = ci1 + ci2+ reg1 +reg2+ vline
    return predictions1, chart


In [40]:
def diff_diff(data, yvar, xvar, year, alpha=0.05):
    import statsmodels.formula.api as smf

    # Grid for predicted values
    data1 = data.loc[(data["policy"]==0) & (data["state"] == 0),:]
    x = data1.loc[pd.notnull(data1[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions1 = pd.DataFrame({xvar: grid})

    # Fit model_before, get predictions
    model_before = smf.ols(f"{yvar} ~ {xvar}", data=data1).fit()
    model_predict = model_before.get_prediction(predictions1[xvar])
    predictions1[yvar] = model_predict.summary_frame()["mean"]
    predictions1[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=alpha)

    # Build chart
    reg1 = alt.Chart(predictions1).mark_line().encode(x=xvar, y=yvar)
    ci1 = (
        alt.Chart(predictions1)
        .mark_errorband()
        .encode(
            x=xvar,
            y=alt.Y("ci_low", title=yvar),
            y2="ci_high",
        )
    )

    data11 = data.loc[(data["policy"]==0) & (data["state"] == 1),:]
    x = data11.loc[pd.notnull(data11[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions11 = pd.DataFrame({xvar: grid})

    # Fit model_before, get predictions
    model_before1 = smf.ols(f"{yvar} ~ {xvar}", data=data11).fit()
    model_predict1 = model_before1.get_prediction(predictions11[xvar])
    predictions11[yvar] = model_predict1.summary_frame()["mean"]
    predictions11[["ci_low", "ci_high"]] = model_predict1.conf_int(alpha=alpha)

    # Build chart
    reg11 = alt.Chart(predictions11).mark_line().encode(x=xvar, y=yvar)
    ci11 = (
        alt.Chart(predictions11)
        .mark_errorband()
        .encode(
            x=xvar,
            y=alt.Y("ci_low", title=yvar),
            y2="ci_high",
        )
    )
    
    data2 = data.loc[(data["policy"]==1) & (data["state"] == 0),:]
    x = data2.loc[pd.notnull(data2[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions2 = pd.DataFrame({xvar: grid})

     # Fit model_before, get predictions
    model_after = smf.ols(f"{yvar} ~ {xvar}", data=data2).fit()
    model_predict_after = model_after.get_prediction(predictions2[xvar])
    predictions2[yvar] = model_predict_after.summary_frame()["mean"]
    predictions2[["ci_low", "ci_high"]] = model_predict_after.conf_int(alpha=alpha)

    # Build chart
    reg2 = alt.Chart(predictions2).mark_line().encode(x=xvar, y=yvar)
    ci2 = (
        alt.Chart(predictions2)
        .mark_errorband()
        .encode(
            x=xvar,
            y=alt.Y("ci_low", title=yvar),
            y2="ci_high",
        )
    )

    data21 = data.loc[(data["policy"]==1) & (data["state"] == 1),:]
    x = data21.loc[pd.notnull(data21[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions21 = pd.DataFrame({xvar: grid})

     # Fit model_before, get predictions
    model_after1 = smf.ols(f"{yvar} ~ {xvar}", data=data21).fit()
    model_predict_after1 = model_after1.get_prediction(predictions21[xvar])
    predictions21[yvar] = model_predict_after1.summary_frame()["mean"]
    predictions21[["ci_low", "ci_high"]] = model_predict_after1.conf_int(alpha=alpha)

    # Build chart
    reg21 = alt.Chart(predictions21).mark_line().encode(x=xvar, y=yvar)
    ci21 = (
        alt.Chart(predictions21)
        .mark_errorband()
        .encode(
            x=xvar,
            y=alt.Y("ci_low", title=yvar),
            y2="ci_high",
        )
    )

    overlay = pd.DataFrame({'x': [year]})
    vline = alt.Chart(overlay).mark_rule(color='red', strokeWidth=3).encode(x='x:Q')
   
    chart = ci1 + ci2+ ci11+ ci21+reg1 +reg11 + reg2+ reg21+vline
    return chart


## Florida

> pre-post opioids

In [41]:
xvar= "TRANSACTION_YEAR"
yvar="opioid_shipment_population_ratio"
fit, chart = pre_post(FL, yvar, xvar, 2010, alpha=0.05)
chart

alt.LayerChart(...)

> diff-diff opioids

In [42]:
xvar= "TRANSACTION_YEAR"
yvar="opioid_shipment_population_ratio"
chart = diff_diff(FL, yvar, xvar, 2010)
chart

alt.LayerChart(...)

> pre-post death rate

In [43]:
xvar= "TRANSACTION_YEAR"
yvar="death_rate"
fit, chart = pre_post(FL, yvar, xvar, 2010, alpha=0.05)
chart

alt.LayerChart(...)

> diff-diff for death rate

In [44]:
xvar= "TRANSACTION_YEAR"
yvar="death_rate"
chart = diff_diff(FL, yvar, xvar, 2010, alpha=0.05)
chart

alt.LayerChart(...)

## Washington

> pre-post opioids

In [45]:
xvar= "TRANSACTION_YEAR"
yvar="opioid_shipment_population_ratio"
fit, chart = pre_post(WA, yvar, xvar, 2012, alpha=0.05)
chart

/opt/miniconda3/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1671: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


alt.LayerChart(...)

> diff-diff opioids

In [46]:
xvar= "TRANSACTION_YEAR"
yvar="opioid_shipment_population_ratio"
chart = diff_diff(WA, yvar, xvar, 2012)
chart

/opt/miniconda3/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1671: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


alt.LayerChart(...)

> pre-post for death rate

In [47]:
xvar= "TRANSACTION_YEAR"
yvar="death_rate"
fit, chart = pre_post(WA, yvar, xvar, 2012, alpha=0.05)
chart

/opt/miniconda3/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1671: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


alt.LayerChart(...)

> diff-diff for death rate

In [48]:
xvar= "TRANSACTION_YEAR"
yvar="death_rate"
chart = diff_diff(WA, yvar, xvar, 2012)
chart

/opt/miniconda3/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1671: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


alt.LayerChart(...)

In [49]:
TX.sample(10)

,BUYER_STATE_x,TRANSACTION_YEAR,population,Deaths,Total shipment,death_rate,opioid_shipment_population_ratio,policy,state
64,TX,2007,23772586.0,1602.0,4.786968e+06,0.000067,0.201365,0,1
44,MD,2014,5887776.0,904.0,2.315904e+06,0.000154,0.393341,1,0
18,GA,2006,9133034.0,412.0,1.978355e+06,0.000045,0.216615,0,0
6,CA,2012,37325068.0,3780.0,1.139486e+07,0.000101,0.305287,1,0
74,UT,2008,2660525.0,354.0,1.082797e+06,0.000133,0.406986,1,0
31,IL,2010,12342797.0,936.0,1.894285e+06,0.000076,0.153473,1,0
5,CA,2011,36969200.0,3948.0,1.098462e+07,0.000107,0.297129,1,0
36,MD,2006,5627367.0,645.0,1.512335e+06,0.000115,0.268746,0,0
10,CO,2007,4795446.0,556.0,1.172815e+06,0.000116,0.244569,0,0
34,IL,2013,12444521.0,1155.0,2.240611e+06,0.000093,0.180048,1,0


## Texas

In [50]:
xvar= "TRANSACTION_YEAR"
yvar="death_rate"
fit, chart = pre_post(TX, yvar, xvar, 2007, alpha=0.05)
chart

/opt/miniconda3/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1671: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


alt.LayerChart(...)

In [51]:
xvar= "TRANSACTION_YEAR"
yvar="death_rate"
chart = diff_diff(TX, yvar, xvar, 2007, alpha=0.05)
chart

/opt/miniconda3/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1671: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


alt.LayerChart(...)